In [120]:
import requests
from bs4 import BeautifulSoup
import re
import time
from urllib.request import urlopen
import random as rm
import unicodedata
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
import pandas as pd
import csv
import json
import numpy as np
import heapq 
import os
import math
import itertools

In [9]:
def save_html(page, index):
    p = requests.get(str(page))
    with open( str(index) + ".html", "w" ) as fh:
        fh.write(p.text)
        
#path = "file:///Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_html/article_9999.html"
def re_fr_dir(path):
    page = urlopen(path)
    movie_soup = BeautifulSoup(page.read().decode("utf-8"), 'html.parser')
    return movie_soup
#movie_soup = re_fr_dir(path)

## Get the list of movies

In [525]:
movies_url = "https://raw.githubusercontent.com/CriMenghini/ADM/master/2019/Homework_3/data/movies1.html"
movies_response = requests.get(movies_url)
print(movies_response)
movies_soup = BeautifulSoup(movies_response.text, 'html.parser')
lst_movies = movies_soup.select("a")
lst_urls = []
for movie in range(0,len(lst_movies)):
    lst_urls.append(lst_movies[movie].get("href")) # get all links of the movies
print('\n'.join(lst_urls[:5]))

<Response [200]>
https://en.wikipedia.org/wiki/Love_by_the_Light_of_the_Moon
https://en.wikipedia.org/wiki/The_Martyred_Presidents
https://en.wikipedia.org/wiki/Terrible_Teddy,_the_Grizzly_King
https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1902_film)
https://en.wikipedia.org/wiki/Alice_in_Wonderland_(1903_film)


## Crawl Wikipedia

In [373]:
# i need to comment it because i dont want to wait again
# for i in range(len(lst_urls)) :
#     save_html(lst_urls[i],"/Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_html/" + "article_" + str(i))
#     if i%10 == 0:
#         time.sleep(int(rm.uniform(1, 5)))    

## Extract all path to the html files

In [3]:
import os
pdir = '/Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_html/'
movies = []
for d, dirs, files in os.walk(pdir):
    for f in files:
        movies.append("file:///Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_html/" + f)

## Extract all information

In [3]:
#extract the plot
def extract_plot_artc(m_soup):
    article = ''
    plot = ''
    pl_soup = m_soup.select(".mw-parser-output > p, .mw-parser-output > h2") # select the class and tags where is plot and intro
    seen_tags = []
    flag = 0
    for pl in pl_soup:
        seen_tags.append(pl.name)
        if seen_tags.count("h2")==0:  # count the tags
            article = article + pl.text # if it is before tag h2 it is intro
        if seen_tags.count("h2")==1 and flag == 1:
            plot = plot + pl.text # if it is after first tag h2 it is plot
        if seen_tags.count("h2")==1:
            flag = 1
        if seen_tags.count("h2")==2:
            break  
    if article == '': # if there is not intro => it is NA
        article = "NA"
    if plot == '': # if there is not plot => it is NA
        plot == 'NA'
    return article, plot

def get_title(m_soup):
    title = re.findall(r".+?(?=\s\D\sWikipedia$)",m_soup.find("title").text)[0] # extract the title and drop the name - Wikipedia
    if title == "": # if there is not title => it is NA
        return "NA"
    else:
        return title
# extract the information in the table
def extract_ch(row):
    res = []
    try:
        for ele in row.find("td").children: # some part of table describet from the tag p
            res.append(ele.text)
    except: 
        try:
            for ele in row.find("td"): # some part of table describet from the tag br which has class 'bs4.element.NavigableString'
                if 'bs4.element.NavigableString' in str(type(ele)):
                    res.append(ele)
        except:
            pass
    return " ".join(res)
# this function account the information
def info(m_soup,d):
    line = m_soup.find("table",{"class": "infobox vevent"}).find('tbody').find_all('tr')
    d["title"] = get_title(movie_soup) # title
    d["intro"],d["plot"] = extract_plot_artc(m_soup)
    flag_page_title = 0
    for row in line: # take the row of table
        try:
            if flag_page_title == 0:
                d["film_name"]  = row.find("th").text
                flag_page_title = 1 # we need this flag to get the name of film
            if row.find("th").text in d.keys():
                d[row.find("th").text] = extract_ch(row) #extract the table info
        except:
            pass
    return d
# remove the stop words and stemming and splitting
def remov_stop_w_and_p_stemm(s):
    stop_words = set(stopwords.words("english")) 
    tok = RegexpTokenizer(r"\w+")
    dirty_words = tok.tokenize(s) # split text
    porter = PorterStemmer()
    stem_words = list(map(porter.stem,dirty_words)) #stemm words
    words = filter(lambda x: x not in string.punctuation, stem_words) # drop punctuation
    cleaned_text = filter(lambda x: x not in stop_words, words) # drop stop words
    return " ".join(list(cleaned_text)) # create the text again

# prepare the dict for scrapping all informations
d = {"title":"","intro":"","plot":"","film_name" : "","Directed by" : "","Produced by" : "","Written by" : "","Starring" : "" ,"Music by" : "", 
     "Release date" : "","Running time" : "","Country": "","Language" : "","Budget" : "","clean_intro":"",
    "clean_plot":"","url":""}
for key,value in d.items():
    if d[key] == "":
        d[key] = "NA"

## Take the info from pages

In [354]:
## vocab = "" # during the extracting information i want to create the vocabulary
# for path in movies:
#     lost_pages = []
#     try:
#         movie_soup = re_fr_dir(path) # take the movie
#         doc = info(movie_soup,d) # take information
#     except:
#         lost_pages.append(path) # catch the expectations
#     for key,value in doc.items():
#         doc[key] = re.sub(r'\n', r"",doc[key]) # from every info block i want to delete the \n and \t
#         doc[key] = re.sub(r'\t', r"",doc[key]) # from every info block i want to delete the \n and \t
#     doc["Release date"] = unicodedata.normalize("NFKD", doc["Release date"]) # normilize the time
#     doc["clean_intro"] = remov_stop_w_and_p_stemm(doc["intro"]) # create the clean intro and plot
#     doc["clean_plot"] = remov_stop_w_and_p_stemm(doc["plot"])
##     vocab = vocab + " " + doc["clean_intro"] + " " + doc["clean_plot"] # creating the vocabulary
#     doc["url"] = str(lst_urls[int(re.findall(r'\d+',re.findall(r'article_.+',path)[0])[0])])
#     file_name = "/Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_tsv/" + re.findall(r'article_.+\.',path)[0] + "tsv"
#     # save the tsv file
#     with open(file_name, 'w', newline='') as f:
#         writer = csv.DictWriter(f, fieldnames = list(doc.keys()), dialect = "excel-tab")
#         writer.writeheader()
#         f.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t" % tuple(doc.values()))
        
    
    
# # #     with open(file_name, "w",encoding="utf-8") as f: 
# # #         f.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t" % tuple(doc.values()))
    
   
    

In [11]:
pdir = '/Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_tsv/'
movies_tsv = []
for d, dirs, files in os.walk(pdir):
    for f in files:
        movies_tsv.append("/Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_tsv/" + f)

In [10]:
def prepare_text(data):
    s1 = ''
    s2 = ''
    if data.clean_intro.isna()[0]: # sometimes there is not plot so we need to prevent it 
        s1 = ''
    else:
        s1 = data.clean_intro.str[:][0]
        
    if data.clean_plot.isna()[0]:
        s2 = ''
    else:
        s2 = data.clean_plot.str[:][0]
    return s1 + " " + s2

## Create the vocabulary

In [14]:
# vocab = ""
# miss_tsv = []
# for path in movies_tsv:
#     #try:
#     df = pd.read_csv(path, sep='\t', index_col= False,encoding='utf-8')
#     try:
#         vocab = vocab + " " + prepare_text(df)
#     except:
#         miss_tsv.append(path)

## Creating the vocab

## Once i need to create and save the vocab

In [20]:
# tok = RegexpTokenizer(r"\w+")
# tru_vocab = set(tok.tokenize(vocab)) # i splited all intro and plot with this function so i have to split all vocab with this vunction
# tru_vocab = (list(tru_vocab))
# vocabulary = {}
# for i in range(len((tru_vocab))):
#     vocabulary[i] = tru_vocab[i]
    

# with open('vocabulary.txt','w') as out:
#     for key,val in vocabulary.items():
#         out.write('{}:{}\n'.format(key,val))

## Upload the vocab

In [8]:
vocabulary = {}
with open('vocabulary.txt') as file:
    for line in file:
        value,key = line.split(":")
        vocabulary[key.strip("\n")] = value

## Create the inverted index

In [25]:
# term_doc = {} # prepare the firs invert index 
# for i in range(len(vocabulary.values())):
#     term_doc[i] = []

# for path in movies_tsv:
#     number = re.findall(r'\d+',path)[1] #number of document
#     df = pd.read_csv(path, sep='\t', index_col= False,encoding='utf-8') # open the doc
#     # take the plot and intro
#     try:
#         clean_intro = df.clean_intro.str[:][0]
#     except:
#         clean_intro = ''
#     try:
#         clean_plot = df.clean_plot.str[:][0]
#     except:
#         clean_plot = ''
#     tok = RegexpTokenizer(r"\w+") 
#     content = list(set(tok.tokenize((clean_intro + " " + clean_plot)))) # take the set of words in document 
    

#     for word in content:
#         term_doc[int(vocabulary[word])].append(int(number))



## Save the Inverted index

In [27]:
# import json
# with open('Inverted_index.json', 'w') as fp:
#     json.dump(term_doc, fp)


## Upload Inverted Index

In [10]:
def jsonKeys2int(x):
    if isinstance(x, dict):
            return {int(k):v for k,v in x.items()}
    return x
Inv_in = json.load(open('Inverted_index.json')) # as we can see now all key are str

Inv_in = jsonKeys2int(Inv_in) # this finction solved the problem with string key

## First Search Engine

In [79]:
def query_upload():
    stop_words = set(stopwords.words("english"))
    tok = RegexpTokenizer(r"\w+")
    porter = PorterStemmer()
    query = tok.tokenize(input()) # take the query and we need to split and stemm it as we sptlited vocab
    query_words = list(map(porter.stem,query)) 
    words = filter(lambda x: x not in string.punctuation, query_words)
    cleaned_query = filter(lambda x: x not in stop_words, words)
    return cleaned_query


In [12]:
query = list(query_upload())

film studio america first motion pictur industri base begin 20th centuri


For ex try this : film studio america first motion pictur industri base begin 20th centuri

## Token the query

In [81]:
def term_query(words,vocabulary):
    res = []
    for i in range(len(words)):
        res.append(int(vocabulary[words[i]]))
    return res

In [14]:
t_query = term_query(query,vocabulary)

## Find the documents which contains all words

In [82]:
def output_search_engine(paths):
    if len(paths) > 1:
        return pd.concat([pd.read_csv(path,sep = "\t", index_col = False,encoding='utf-8')[["title","intro","url"]] for path in paths]).reset_index(drop = True)
    else:
        return pd.concat([pd.read_csv(path,sep = "\t", index_col = False,encoding='utf-8')[["title","intro","url"]] for path in paths[:1]]).reset_index(drop = True)
    return res
def Seqrch_Engine1(query,inver_in):
    docs = []
    for term in query:
        docs.append(inver_in[term])
    for i in range(len(docs)): # from list to sets
        docs[i] = set(docs[i])
    res  = docs[0].intersection(*[i for i in docs[:]])
    if len(res) != 0 :
        paths = []
        for i in res:
            paths.append("/Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_tsv/article_" + str(i) + ".tsv")
        output = output_search_engine(paths)
        
        return output_search_engine(paths)
            
    else:
        return print("There is not any movie with ALL whis words")
    return res


In [16]:
Seqrch_Engine1(t_query,Inv_in)

,title,intro,url
0,The Musketeers of Pig Alley,The Musketeers of Pig Alley is a 1912 American...,https://en.wikipedia.org/wiki/The_Musketeers_o...
1,The Black Viper,The Black Viper (aka La vipère noire in France...,https://en.wikipedia.org/wiki/The_Black_Viper
2,An Unseen Enemy,An Unseen Enemy is a 1912 Biograph Company sho...,https://en.wikipedia.org/wiki/An_Unseen_Enemy
3,The Call of the Wild (1908 film),The Call of the Wild is a 1908 American short ...,https://en.wikipedia.org/wiki/The_Call_of_the_...
4,The Poor Little Rich Girl,The Poor Little Rich Girl is a 1917 American c...,https://en.wikipedia.org/wiki/The_Poor_Little_...
5,At the Altar,At the Altar is a 1909 American silent drama f...,https://en.wikipedia.org/wiki/At_the_Altar
6,The Venus Model,The Venus Model is a 1918 American silent roma...,https://en.wikipedia.org/wiki/The_Venus_Model


## Prepare for cosine similarity

In [17]:
import math
number_of_documents = len(movies_tsv) # the number of document in which we will search the movie
def idf(Inv_in,number_of_documents):
    term_idf = {}
    # idf = log2(number of document / number of doc consists the term key )
    lost_words = []
    for key,values in Inv_in.items(): 
        try:
            term_idf[key] = math.log(number_of_documents/len(values),10) #logarith by 10
        except:
            lost_words.append(key)
    return lost_words,term_idf

In [18]:
lost_words,term_idf = idf(Inv_in,number_of_documents) # create the idf for every file

In [83]:
def split_text(s):
    tok = RegexpTokenizer(r"\w+")
    words = tok.tokenize(s)
    return words

def find_word (vocab,word_id):
    for word,term in vocab.items():
        if int(term) == word_id:
            break 
    return word
def prepare_text(data):
    s1 = ''
    s2 = ''
    if data.clean_intro.isna()[0]: # sometimes there is not plot so we need to prevent it 
        s1 = ''
    else:
        s1 = data.clean_intro.str[:][0]
        
    if data.clean_plot.isna()[0]:
        s2 = ''
    else:
        s2 = data.clean_plot.str[:][0]
        
    return s1 + " " + s2
def word_to_term(lst_words , vocab):
    terms = []
    for word in lst_words:
        try:
            terms.append(int(vocab[word]))
        except:
            pass
    
    return terms 

def count_length(terms,idf_ls,len_document):
    lenght = 0
    for term in terms:
        lenght += (idf_ls[term]/len_document)**2
    
    return np.sqrt(lenght)

## Create the Inverted Index 2.1

In [38]:
# inv_ind2 = {}
# doc_lenght = {}
# for key,value in Inv_in.items():
#     inv_ind2[key] = []


# for path in movies_tsv:
#     doc_num = int(re.findall(r'\d+',re.findall(r'article_.+',path)[0])[0])
#     df = pd.read_csv(path,sep = "\t", index_col = False,encoding='utf-8')[["clean_intro","clean_plot"]] # open the file with DataFrame
#     text = prepare_text(df) # prepare the text to avoid any NaN
#     words = split_text(text)
#     terms = word_to_term(words , vocabulary)
#     doc_lenght[doc_num] = count_length(terms,term_idf,len(terms))
#     d_length = len(terms)
#     unic_terms = list(set(terms))

#     for t in unic_terms:    
#         freq = terms.count(t)
#         td_idf = freq * term_idf[t] # idf of term multiply on td
#         inv_ind2[t].append((doc_num, td_idf/d_length))
  

In [39]:
# #as we can understand the length of inverted index have to be tha same so as we can see there are the same it means that everithing is okey
# a = []
# for key1,value1 in inv_ind2.items():
#     if len(Inv_in[key1]) == len(value1):
#          pass
#     else:
#         a.append(key1)
    
# import json
# with open('Inverted_index_2_1.json', 'w') as fp:
#     json.dump(inv_ind2, fp)
    
# with open('doc_lenght.json', 'w') as fp:
#     json.dump(doc_lenght, fp)           

## Save inverted index 2.1 and length of documents

In [41]:
# import json
# with open('Inverted_index_2_1.json', 'w') as fp:
#     json.dump(inv_ind2, fp)
    
# with open('doc_lenght.json', 'w') as fp:
#     json.dump(doc_lenght, fp)

## Upload inverted index 2.1 and length of docs

In [20]:
inv_ind2 = json.load(open('Inverted_index_2_1.json')) # as we can see now all key are str
inv_ind2 = jsonKeys2int(inv_ind2) # this finction solved the problem with string key
for key in inv_ind2.keys(): # after uploading json all tuples became lists 
    for i in range(len(inv_ind2[key])):
        inv_ind2[key][i] = tuple(inv_ind2[key][i]) # turn back from lists to tuples
for key in inv_ind2.keys():
    for i in range(len(inv_ind2[key])):
        inv_ind2[key][i] = tuple(inv_ind2[key][i])
doc_lenght = json.load(open('doc_lenght.json')) # as we can see now all key are str
doc_lenght = jsonKeys2int(doc_lenght)

## Quiery

In [117]:
def quiery_tdidf(q,idf):
    q_td_idf = {}
    # idf = log2(number of document / number of doc consists the term key )
    for term in q:
        q_td_idf[term] = idf[term]/len(q) # quiery tdidf (td = 1/len(document))
    return q_td_idf 

def Search_engine_2_all_docs(query,vocabulary,idf,inv_in):
    t_query = term_query(query,vocabulary) # take the words and create the string
    lenght_q = count_length(t_query,idf,len(t_query))
    q_td_idf = quiery_tdidf(t_query,term_idf)
    terms_in_docs = []
    cossim = {}
    d = {}
    for term in t_query:
        terms_in_docs.append((term,inv_in[term]))
    
    for info in terms_in_docs:
        term = info[0]
        for doc_tdidf in info[1]:
            if doc_tdidf[0] in d.keys(): 
                d[doc_tdidf[0]].append(doc_tdidf[1] * q_td_idf[term])
            elif doc_tdidf[0] not in d.keys():
                d[doc_tdidf[0]] = []
                d[doc_tdidf[0]].append(doc_tdidf[1] * q_td_idf[term])
        for num_doc in d.keys():
            cossim[str(num_doc)] = sum(d[num_doc])/(doc_lenght[num_doc]*lenght_q ) # count the cosSim

    return cossim

# this function works like first search engine but the output is the list of documents which conteins all words from the quaery
def find_all_docs_consist_all_quiery(query,inver_in):
    docs = []
    for term in query:
        docs.append(inver_in[term])
    for i in range(len(docs)): # from list to sets
        docs[i] = set(docs[i])
    res  = docs[0].intersection(*[i for i in docs[:]])
    if len(res) != 0 :
        paths = []
        for i in res:
            paths.append("/Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_tsv/article_" + str(i) + ".tsv")
        return paths
            
    else:
        return [] # if there if not such documents
    return res
# Search engine 2
def Search_engine_2(query,vocabulary,idf,inv_in,paths_doc):
    terms_in_docs = []
    cossim = {}
    d = {}
    for key in paths_doc:
        d[int(re.findall(r'\d+',re.findall(r'article_.+',key)[0])[0])] = [] #prepare the dict of the documents which contains all words from the query
    # prepare the query    
    t_query = term_query(query,vocabulary) # take the words and create the string
    lenght_q = count_length(t_query,idf,len(t_query)) # count the norm of the query
    q_td_idf = quiery_tdidf(t_query,term_idf) # return tdidf query
    #prepare
    for term in t_query:
        terms_in_docs.append((term,inv_in[term])) #take the inverted index only for that terms which is in the query
    # take the term
    for info in terms_in_docs:
        term = info[0] # take the term
        for doc_tdidf in info[1]: #take the number of document
            if doc_tdidf[0] in d.keys(): # if the number of doc in the prepared list of documents which consists all words from the query
                d[doc_tdidf[0]].append(doc_tdidf[1] * q_td_idf[term]) # count the numerator cosine similarity

    for num_doc in d.keys():
        cossim[str(num_doc)] = sum(d[num_doc])/(doc_lenght[num_doc]*lenght_q ) # count the cosine simimilarity

    return cossim
# this function printing creating the top k movies
def output_search_engine(paths,top_k):
    if len(paths) > 1:
        a = pd.concat([pd.read_csv(path,sep = "\t", index_col = False)[["title","intro","url"]] for path in paths]).reset_index(drop = True)
        a['Similarity'] = top_k
        return a
    else:
        a = pd.concat([pd.read_csv(path,sep = "\t", index_col = False)[["title","intro","url"]] for path in paths]).reset_index(drop = True)
        a['Similarity'] = top_k
        return a
# this function use the HEAP structure to find the top k movies
def top_k(k,cossim):
    heap = [(value, key) for key,value in cossim.items()] # to do a heap from the dict we need as key the score of movies
    largest = heapq.nlargest(k, heap) #get top k movies 
    paths = []
    # take the paths of top k movies
    paths = ["/Users/macbook/Desktop/Lev/Sapienza/ADM_Labs/HW3/data_tsv/article_" + str(num_doc[1]) + ".tsv" for num_doc in largest]
    # print(paths)
    return output_search_engine(paths,[i for i,j in largest]) # print the top k movies


In [22]:
query = list(query_upload()) #take the string and create the words
t_query = term_query(query,vocabulary)
paths_doc = find_all_docs_consist_all_quiery(t_query,Inv_in) # i use first search engine to find all documents which consist all quaery
if len(paths_doc) == 0: # if there is not documents which contains all words from the query
    cossim = Search_engine_2_all_docs(query,vocabulary,term_idf,inv_ind2) # this function will count the cossine similarity for all documents which consists at least one wod from the query
else:
    cossim = Search_engine_2(query,vocabulary,term_idf,inv_ind2,paths_doc)

star future space


Example query: star future space

## Print top k = 5 movies

In [23]:
k = 5
top_k(k,cossim)

,title,intro,url,Similarity
0,The Jetsons Meet the Flintstones,The Jetsons Meet the Flintstones is a 1987 ani...,https://en.wikipedia.org/wiki/The_Jetsons_Meet...,0.588534
1,Lost in Space (film),Lost in Space is a 1998 American science-fict...,https://en.wikipedia.org/wiki/Lost_in_Space_(f...,0.303305
2,Zathura: A Space Adventure,Zathura: A Space Adventure (also known simply...,https://en.wikipedia.org/wiki/Zathura_(film),0.267976
3,The Phantom Planet,The Phantom Planet is a 1961 independently mad...,https://en.wikipedia.org/wiki/The_Phantom_Planet,0.254209
4,Predestination (film),Predestination is a 2014 Australian science f...,https://en.wikipedia.org/wiki/Predestination_(...,0.195048


I decided to use top 5 because i suppose that 5 option of the movies is enough to 

## Search engine 3

The idia is to add the title to the vocabulary. Also i want to use a little bit different tdidf weight scheme,
I decided to improve the idf. The formula for search engine 2 idf = log(N/N_t) which means that if in every document we have one tearm we will get log(1) = 0, so to prevent it i decided to use formula idf = log(1 + N/N_t) which means that the lower bound of the terms will be log(2).

In [31]:
def prep_ti_text(data):
    s1 = ''
    s2 = ''
    s3 = ''
    if data.clean_intro.isna()[0]: # sometimes there is not plot so we need to prevent it 
        s1 = ''
    else:
        s1 = data.clean_intro.str[:][0]
    if data.clean_plot.isna()[0]:
        s2 = ''
    else:
        s2 = data.clean_plot.str[:][0]
    if data.title.isna()[0]: 
        s3 = ''
    else:
        s3 = data.title.str[:][0]
    return s1 + " " + s2 + " " + s3

In [32]:
# vocab = ""
# miss_tsv = []
# for path in movies_tsv:
#     #try:
#     df = pd.read_csv(path, sep='\t', index_col= False,encoding='utf-8')
#     try:
#         vocab = vocab + " " + prepare_text(df)
#     except:
#         miss_tsv.append(path)
# tok = RegexpTokenizer(r"\w+")
# tru_vocab = set(tok.tokenize(vocab)) # i splited all intro and plot with this function so i have to split all vocab with this vunction
# tru_vocab = (list(tru_vocab))
# vocabulary = {}
# for i in range(len((tru_vocab))):
#     vocabulary[i] = tru_vocab[i]
    

In [33]:
# with open('vocabulary_search3.txt','w') as out:
#     for key,val in vocabulary.items():
#         out.write('{}:{}\n'.format(key,val))

## Upload the inverted vocabular with the titles

In [34]:
vocabulary = {}
with open('vocabulary_search3.txt') as file:
    for line in file:
        value,key = line.split(":")
        vocabulary[key.strip("\n")] = value

In [35]:
# term_doc = {} # prepare the firs invert index 
# for i in range(len(vocabulary.values())):
#     term_doc[i] = []

# for path in movies_tsv:
#     number = re.findall(r'\d+',path)[1] #number of document
#     df = pd.read_csv(path, sep='\t', index_col= False,encoding='utf-8') # open the doc
#     # take the plot and intro
#     try:
#         clean_intro = df.clean_intro.str[:][0]
#     except:
#         clean_intro = ''
#     try:
#         clean_plot = df.clean_plot.str[:][0]
#     except:
#         clean_plot = ''
#     tok = RegexpTokenizer(r"\w+") 
#     content = list(set(tok.tokenize((clean_intro + " " + clean_plot)))) # take the set of words in document 
    

#     for word in content:
#         term_doc[int(vocabulary[word])].append(int(number))



In [36]:
# import json
# with open('Inverted_index_3.json', 'w') as fp:
#     json.dump(term_doc, fp)


## Upload the inverted_index

In [37]:
def jsonKeys2int(x):
    if isinstance(x, dict):
            return {int(k):v for k,v in x.items()}
    return x
Inv_in = json.load(open('Inverted_index_3.json')) # as we can see now all key are str

Inv_in = jsonKeys2int(Inv_in) # this finction solved the problem with string key

In [54]:
number_of_documents = len(movies_tsv) # the number of document in which we will search the movie
def idf(Inv_in,number_of_documents):
    term_idf = {}
    # idf = log2(number of document / number of doc consists the term key )
    lost_words = []
    for key,values in Inv_in.items(): 
        try:
            term_idf[key] = math.log(1 + (number_of_documents/len(values)),10) # smoothing the tdidf score
        except:
            lost_words.append(key)
    return lost_words,term_idf

In [55]:
lost_words,term_idf = idf(Inv_in,number_of_documents) # create the idf for every file

In [62]:
def prep_ti_text(data):
    s1 = ''
    s2 = ''
    s3 = ''
    if data.clean_intro.isna()[0]: # sometimes there is not plot so we need to prevent it 
        s1 = ''
    else:
        s1 = data.clean_intro.str[:][0]
    if data.clean_plot.isna()[0]:
        s2 = ''
    else:
        s2 = data.clean_plot.str[:][0]
    if data.title.isna()[0]: 
        s3 = ''
    else:
        s3 = data.title.str[:][0]
    return s1 + " " + s2 + " " + s3

Also the inverted index is changing because now we work with the title too, and i use smooth because as we can see during the scraping in the title always was the word "Wikipedia" and if we add the title to the plot and intro we can understand that with the old idf formula we would get the idf for word title = 0 but now we will have at least log(2). So if someone will want to find the movie by the word "Wikipedia" there still will have top movies.

In [70]:
# inv_ind2 = {}
# doc_lenght = {}
# for key,value in Inv_in.items():
#     inv_ind2[key] = []


# for path in movies_tsv:
#     doc_num = int(re.findall(r'\d+',re.findall(r'article_.+',path)[0])[0])
#     df = pd.read_csv(path,sep = "\t", index_col = False,encoding='utf-8')[["title","clean_intro","clean_plot"]] # open the file with DataFrame
#     text = prep_ti_text(df) # prepare the text with title, plot, intro
#     words = split_text(text)
#     terms = word_to_term(words , vocabulary)
#     doc_lenght[doc_num] = count_length(terms,term_idf,len(terms))
#     d_length = len(terms)
#     unic_terms = list(set(terms))

#     for t in unic_terms:    
#         freq = terms.count(t)
#         td_idf = freq * term_idf[t] # idf of term multiply on td
#         inv_ind2[t].append((doc_num, td_idf/d_length))
  


In [74]:
# import json
# with open('Inverted_index_3.json', 'w') as fp:
#     json.dump(inv_ind2, fp)
    
# with open('doc_lenght_3.json', 'w') as fp:
#     json.dump(doc_lenght, fp)    


## Upload the inverted index 3

In [76]:
inv_ind3 = json.load(open('Inverted_index_3.json')) # as we can see now all key are str
inv_ind3 = jsonKeys2int(inv_ind2) # this finction solved the problem with string key
for key in inv_ind2.keys(): # after uploading json all tuples became lists 
    for i in range(len(inv_ind2[key])):
        inv_ind2[key][i] = tuple(inv_ind2[key][i]) # turn back from lists to tuples
for key in inv_ind2.keys():
    for i in range(len(inv_ind2[key])):
        inv_ind2[key][i] = tuple(inv_ind2[key][i])
doc_lenght = json.load(open('doc_lenght_3.json')) # as we can see now all key are str
doc_lenght = jsonKeys2int(doc_lenght)

we can use the function for second search engine but with different idf the results will be different 

In [111]:
query = list(query_upload()) #take the string and create the words
t_query = term_query(query,vocabulary)
paths_doc = find_all_docs_consist_all_quiery(t_query,Inv_in) # i use first search engine to find all documents which consist all quaery
if len(paths_doc) == 0: # if there is not documents which contains all words from the query
    cossim = Search_engine_2_all_docs(query,vocabulary,term_idf,inv_ind3) # this function will count the cossine similarity for all documents which consists at least one wod from the query
else:
    cossim = Search_engine_2(query,vocabulary,term_idf,inv_ind3,paths_doc)

star future space


as we can see the cosine similarity has changed comparing with the old score

In [112]:
k = 5
top_k(k,cossim)

,title,intro,url,Similarity
0,The Jetsons Meet the Flintstones,The Jetsons Meet the Flintstones is a 1987 ani...,https://en.wikipedia.org/wiki/The_Jetsons_Meet...,0.578095
1,Lost in Space (film),Lost in Space is a 1998 American science-fict...,https://en.wikipedia.org/wiki/Lost_in_Space_(f...,0.302463
2,Zathura: A Space Adventure,Zathura: A Space Adventure (also known simply...,https://en.wikipedia.org/wiki/Zathura_(film),0.263370
3,The Phantom Planet,The Phantom Planet is a 1961 independently mad...,https://en.wikipedia.org/wiki/The_Phantom_Planet,0.250881
4,Predestination (film),Predestination is a 2014 Australian science f...,https://en.wikipedia.org/wiki/Predestination_(...,0.192607


## The 4th question

In [5]:
def MaxLengthPalindrome(S):
    combosList  = list(itertools.chain.from_iterable([list(itertools.combinations(S,i)) for i in range(1,len(S))])) #list form
    combosString = [''.join(c) for c in combosList]  #string form of every possibile subsequence 
    chk=0
    longest=0
    for sub in combosString:     #for every subsequence
            for k in range(len(sub)):   
                if(sub[k]==sub[len(sub)-1-k]): #we test if first element is equal to last element(second with second-last and so on)
                    chk+=1
                    if(chk==len(sub)):  #if true we found a palindrome
                        if(len(sub)>longest): 
                            longest=len(sub)   #we save the longest
                else:
                    chk=0
                    break
    return print('The longest Palindrome subsequence is %s ' %longest)

In [6]:
MaxLengthPalindrome('DATAMININGSAPIENZA')

The longest Palindrome subsequence is 7 
